Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [X] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [X] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [21]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [22]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [23]:
import numpy as np
import pandas as pd

def wrangle(X):
  #Prevent SettingWithCopy Warning :
  X=X.copy()
  #Replace near zero values with zero in lattitude
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  #Replacing zeros with nulls for required columns
  #create a missing indicator column, might become handy 
  cols_with_zero = ['longitude', 'latitude', 'construction_year','gps_height',
                    'population']
  for col in cols_with_zero:
    X[col] = X[col].replace(0, np.nan)
    X[col+'_missing'] = X[col].isnull()
  #drop duplicate columns
  duplicates = ['quantity_group', 'payment_type']
  X=X.drop(columns=duplicates)
  #convert date_recorded to datetime format
  X['date_recorded']=pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
  #Create Year, month, day columns and drop the original column
  X['year_recorded']=X['date_recorded'].dt.year
  X['month_recorded']=X['date_recorded'].dt.month
  X['day_recorded']=X['date_recorded'].dt.day
  X=X.drop(columns='date_recorded')
  #Engineer feature: years from construction to recorded
  X['years_construction_recorded'] = X['year_recorded']-X['construction_year']
  X['years_missing']= X['years_construction_recorded'].isnull()

  return X 

train = wrangle(train)
test = wrangle(test)

train.shape , test.shape

((59400, 48), (14358, 47))

In [24]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# set the target
target = 'status_group'
# drop target from train
train_features = train.drop(columns=target)
# create a list of numeric columns 
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
# get a series with cardinality of non numeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()
# create a list of non numeric features with less than 50 cardinality
catagorical_features = cardinality.index.tolist()
# combine lists 
features = numeric_features+catagorical_features

X_train = train[features]
y_train = train[target]

pipeline = make_pipeline(ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(random_state=42, n_jobs=-1))

k = 100 

scores = cross_val_score(pipeline, X_train, y_train, cv=k, scoring='accuracy')

scores

/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/py

/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/py

/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/amin/opt/anaconda3/lib/py

array([0.80704698, 0.77684564, 0.79194631, 0.81711409, 0.80033557,
       0.79865772, 0.78355705, 0.80536913, 0.79362416, 0.81711409,
       0.78691275, 0.79697987, 0.8204698 , 0.80369128, 0.78691275,
       0.77348993, 0.79865772, 0.84705882, 0.80840336, 0.79327731,
       0.77647059, 0.8       , 0.78991597, 0.83529412, 0.8030303 ,
       0.78787879, 0.79124579, 0.78619529, 0.79292929, 0.79292929,
       0.81986532, 0.78282828, 0.78451178, 0.77272727, 0.78619529,
       0.78619529, 0.81144781, 0.79461279, 0.80976431, 0.76936027,
       0.80639731, 0.78619529, 0.81313131, 0.81649832, 0.82828283,
       0.78114478, 0.7979798 , 0.7962963 , 0.79461279, 0.81144781,
       0.7979798 , 0.78619529, 0.7996633 , 0.8047138 , 0.78114478,
       0.8013468 , 0.7979798 , 0.79461279, 0.78282828, 0.80269815,
       0.78752108, 0.80775717, 0.79595278, 0.81787521, 0.8010118 ,
       0.80607083, 0.78752108, 0.80269815, 0.80775717, 0.79089376,
       0.78752108, 0.79426644, 0.79932546, 0.7925801 , 0.81618

In [25]:
scores.mean()

0.7971698620036602

In [26]:
X_train 

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,...,source_type,source_class,waterpoint_type,waterpoint_type_group,longitude_missing,latitude_missing,construction_year_missing,gps_height_missing,population_missing,years_missing
0,69572,6000.0,1390.0,34.938093,-9.856322,0,11,5,109.0,1999.0,...,spring,groundwater,communal standpipe,communal standpipe,False,False,False,False,False,False
1,8776,0.0,1399.0,34.698766,-2.147466,0,20,2,280.0,2010.0,...,rainwater harvesting,surface,communal standpipe,communal standpipe,False,False,False,False,False,False
2,34310,25.0,686.0,37.460664,-3.821329,0,21,4,250.0,2009.0,...,dam,surface,communal standpipe multiple,communal standpipe,False,False,False,False,False,False
3,67743,0.0,263.0,38.486161,-11.155298,0,90,63,58.0,1986.0,...,borehole,groundwater,communal standpipe multiple,communal standpipe,False,False,False,False,False,False
4,19728,0.0,NaN,31.130847,-1.825359,0,18,1,NaN,NaN,...,rainwater harvesting,surface,communal standpipe,communal standpipe,False,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,1210.0,37.169807,-3.253847,0,3,5,125.0,1999.0,...,spring,groundwater,communal standpipe,communal standpipe,False,False,False,False,False,False
59396,27263,4700.0,1212.0,35.249991,-9.070629,0,11,4,56.0,1996.0,...,river/lake,surface,communal standpipe,communal standpipe,False,False,False,False,False,False
59397,37057,0.0,NaN,34.017087,-8.750434,0,12,7,NaN,NaN,...,borehole,groundwater,hand pump,hand pump,False,False,True,True,True,True
59398,31282,0.0,NaN,35.861315,-6.378573,0,1,4,NaN,NaN,...,shallow well,groundwater,hand pump,hand pump,False,False,True,True,True,True


In [27]:
y_train

0            functional
1            functional
2            functional
3        non functional
4            functional
              ...      
59395        functional
59396        functional
59397        functional
59398        functional
59399        functional
Name: status_group, Length: 59400, dtype: object

In [29]:
from scipy.stats import randint,uniform
from sklearn.model_selection import RandomizedSearchCV

features = numeric_features+catagorical_features
X_train = train[features]
y_train = train[target]

pipeline = make_pipeline(ce.OrdinalEncoder(),
                        SimpleImputer(),
                        RandomForestClassifier(random_state=42)
                        )

param_distributions = {'simpleimputer__strategy':['mean','median'],
                      'randomforestclassifier__n_estimators':randint(50,500),
                      'randomforestclassifier__max_depth':[5, 10, 15, 20, None],
                      'randomforestclassifier__max_features': uniform(0,1),
                       'randomforestclassifier__min_samples_split': range(5,50),
                       'randomforestclassifier__min_samples_leaf': randint(1,1000),
                      }

search = RandomizedSearchCV(pipeline,
                           param_distributions=param_distributions,
                           n_iter=100,
                           cv=5,
                           scoring='accuracy',
                           verbose=10,
                           return_train_score=True,
                           n_jobs=-1)

search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 19.7min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 21.7min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 23.9min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 27.3min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 32

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            f

In [30]:
print('Best hyperparameters:', search.best_params_)
print('Cross Validation Score:', search.best_score_)

Best hyperparameters: {'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 0.6100146022491, 'randomforestclassifier__min_samples_leaf': 20, 'randomforestclassifier__min_samples_split': 9, 'randomforestclassifier__n_estimators': 333, 'simpleimputer__strategy': 'mean'}
Cross Validation Score: 0.7927777777777778


In [31]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,99,22,53,17,84,35,81,57,74,75,...,25,62,20,66,95,39,82,19,91,14
mean_fit_time,898.477,142.898,51.7275,196.283,172.514,201.166,55.4168,89.997,55.972,232.731,...,16.0407,24.4927,7.75039,22.624,17.4594,15.3345,8.07409,12.6101,6.14413,11.0105
std_fit_time,787.531,2.08958,0.759739,3.17473,61.6469,4.41463,1.24978,2.48174,0.253831,0.4855,...,0.104531,0.698305,0.14221,4.93865,0.0625378,0.108388,0.0967282,0.0609307,0.0427108,0.157899
mean_score_time,2.88851,1.1095,0.850826,1.26626,2.42438,1.2103,0.564703,0.88226,0.993934,1.16627,...,0.832331,1.04716,0.555032,0.982789,0.94247,1.21886,0.698342,0.994018,0.592251,0.961516
std_score_time,3.55933,0.0991842,0.0557035,0.0208164,3.03971,0.055814,0.0115373,0.0660005,0.00823207,0.0272274,...,0.0384839,0.0109067,0.0156038,0.0661561,0.0125201,0.0499079,0.0148645,0.0187094,0.00648696,0.0381115
param_randomforestclassifier__max_depth,None,None,15,20,15,None,15,15,10,10,...,20,10,20,10,5,None,None,20,5,10
param_randomforestclassifier__max_features,0.610015,0.665553,0.332861,0.698054,0.935284,0.75931,0.811593,0.550435,0.257702,0.916342,...,0.0901844,0.102124,0.0803672,0.0837652,0.0744876,0.040774,0.030969,0.00161183,0.0123347,0.0290087
param_randomforestclassifier__min_samples_leaf,20,22,36,72,66,98,107,110,59,77,...,703,799,627,788,133,180,373,697,632,991
param_randomforestclassifier__min_samples_split,9,49,22,26,19,9,22,14,22,18,...,47,46,41,7,21,31,28,27,11,23
param_randomforestclassifier__n_estimators,333,235,177,374,194,367,103,232,321,438,...,260,432,104,391,446,459,240,451,179,432
param_simpleimputer__strategy,mean,mean,mean,mean,median,median,mean,mean,mean,mean,...,mean,median,median,mean,mean,mean,median,median,mean,mean


In [32]:
pipeline = search.best_estimator_

In [33]:
X_test = test[features]
y_pred = pipeline.predict(X_test)

In [ ]:
submission = test[['id']].copy()
submission['status_group']=y_pred
submission.to_csv('submission03.csv', index=False)